In [2]:
import openai
import os
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# Remember that you need to set the OPENAI_API_BASE to point openai to your specific deployment.

openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

Now try to perform a streaming api question to chatgpt.
First of all one simple question without streaming.

In [3]:
from pprint import pprint
response = openai.ChatCompletion.create(
  engine="Gpt35",
  messages = [{"role":"system",
               "content":"You are an AI assistant that helps people find information."},
               {"role":"user","content":"Tells me top 10 owasp vulnerabilities for web app with a small explanation for each one"}
  ],
  max_tokens=1600,
  temperature=0.3,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

pprint(response)
pprint(response["choices"][0]["message"]["content"])

<OpenAIObject chat.completion id=chatcmpl-7TsynmKwwgnx1pAQqdBvu5D3bLDAi at 0x7fe925c72840> JSON: {
  "id": "chatcmpl-7TsynmKwwgnx1pAQqdBvu5D3bLDAi",
  "object": "chat.completion",
  "created": 1687357209,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Sure, here are the top 10 OWASP vulnerabilities for web applications along with a brief explanation for each one:\n\n1. Injection: Injection flaws occur when an attacker can send untrusted data to an interpreter that can execute unintended commands or access unauthorized data. This can result in data loss, corruption, or unauthorized access to sensitive information.\n\n2. Broken Authentication and Session Management: This vulnerability occurs when an application does not properly authenticate users or manage their sessions. Attackers can exploit this vulnerability to gain unauthorized access to user accounts or sensitive d

You can notice that the answer comes all at a time, the server will perform the response then it answer with a complete result.

To perform a **streaming request** you need to change the code a little, with this method you can print a little bit of a response at a time.

In [10]:
import pprint
# create a for to enumerate the chunks returned by the chat api
# pp = pprint.PrettyPrinter(width=80)

for chunk in openai.ChatCompletion.create(
  engine="Gpt35",
  messages = [{"role":"system",
               "content":"You are an AI assistant that helps people find information."},
               {"role":"user","content":"Tells me top 10 owasp vulnerabilities for web app with a small explanation for each one"}
  ],
  max_tokens=1600,
  temperature=0.3,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None,
  stream=True):
    # this is the inside of the for cycle, lets print the chunk
    # if you print the whole chunk it will print gobs of text
    # pprint(chunk)
    delta = chunk["choices"][0].get("delta", {}).get("content")
    if delta is not None:
        print(delta, end="")


Sure, here are the top 10 OWASP vulnerabilities for web applications along with a brief explanation for each one:

1. Injection: Injection flaws occur when untrusted data is sent to an interpreter as part of a command or query. This can result in the execution of unintended commands or access to unauthorized data.

2. Broken Authentication and Session Management: This vulnerability occurs when an application fails to properly authenticate users or manage their sessions, allowing attackers to gain unauthorized access to sensitive data.

3. Cross-Site Scripting (XSS): XSS vulnerabilities allow attackers to inject malicious scripts into web pages viewed by other users. This can result in the theft of sensitive data or the execution of unauthorized actions on behalf of the victim.

4. Broken Access Control: This vulnerability occurs when an application fails to properly enforce access controls, allowing attackers to access sensitive data or functionality that should be restricted.

5. Secu

Now you can also using async python

In [11]:
import pprint
# create a for to enumerate the chunks returned by the chat api
# pp = pprint.PrettyPrinter(width=80)
# !pip install asyncio

async for chunk in await openai.ChatCompletion.acreate(
  engine="Gpt35",
  messages = [{"role":"system",
               "content":"You are an AI assistant that helps people find information."},
               {"role":"user","content":"Tells me top 10 owasp vulnerabilities for web app with a small explanation for each one"}
  ],
  max_tokens=1600,
  temperature=0.3,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None,
  stream=True):
    # this is the inside of the for cycle, lets print the chunk
    # if you print the whole chunk it will print gobs of text
    # pprint(chunk)
    delta = chunk["choices"][0].get("delta", {}).get("content")
    if delta is not None:
        print(delta, end="")


Sure, here are the top 10 OWASP vulnerabilities for web applications:

1. Injection: Injection flaws, such as SQL, NoSQL, OS, and LDAP injection, occur when untrusted data is sent to an interpreter as part of a command or query. The attacker's hostile data can trick the interpreter into executing unintended commands or accessing data without proper authorization.

2. Broken Authentication and Session Management: Applications frequently fail to protect sensitive information, such as authentication credentials and session tokens, due to improper authentication and session management.

3. Cross-Site Scripting (XSS): XSS flaws occur whenever an application includes untrusted data in a new web page without proper validation or escaping, allowing attackers to execute scripts in the victim's browser that can hijack user sessions, deface web sites, or redirect the user to malicious sites.

4. Broken Access Control: Restrictions on what authenticated users are allowed to do are not properly enf